# Process @jongraz tiktok stats for Noodle videos

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read in our hand collected list of Noodle videos

In [3]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Create an ID field from the URL

In [4]:
src['id'] = src['url'].str.replace('https://www.tiktok.com/@jongraz/video/', "", regex=False).astype(str)

### Make a list for subsetting the dataframe later

In [5]:
bones_videos = src['id'].to_list()

### Have to run this manually -- for now

In [6]:
# Get cookie and run this...

In [7]:
# !python '/Users/stiles/github/noodle-tracker/_workspace/grab_user_stats.py'

---

### Read scraped stats data

In [8]:
df = pd.read_csv('data/processed/jongraz_videos.csv', dtype={'video_id':str})

### Clean update dates, times

In [9]:
df['timestamp'] = pd.to_datetime(df['video_time'], unit='s')
df['date'] = pd.to_datetime(df['video_time'], unit='s').dt.date
df['month'] = pd.to_datetime(df['video_time'], unit='s').dt.month
df['weekday'] = pd.to_datetime(df['video_time'], unit='s').dt.day_name()
df['time'] = pd.to_datetime(df['video_time'], unit='s').dt.time
df['weekstart_timestamp'] = pd.to_datetime(df['video_time'], unit='s') - pd.to_datetime(df['video_time'], unit='s').dt.weekday * np.timedelta64(1, 'D')
df['weekstart'] = df['weekstart_timestamp'].dt.date
df['updated_date'] = pd.to_datetime("today")

### How many videos? 

In [10]:
len(df)

214

### Select only "bones" videos from @jongraz's feed

In [11]:
bones_df = df[df['video_id'].isin(bones_videos)]

### How many are Noodle videos? 

In [12]:
len(bones_df)

91

In [13]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
2,jongraz,7083871650818641198,beached and unbothered,1649342397,15,1895,68300,878,335500,https://www.tiktok.com/@jongraz/video/7083871650818641198?lang=en,2022-04-07 14:39:57,2022-04-07,4,Thursday,14:39:57,2022-04-04 14:39:57,2022-04-04,2022-04-10 10:53:29.004647
6,jongraz,7083127893210991915,another day another glaring truth,1649169228,30,1523,54700,556,270900,https://www.tiktok.com/@jongraz/video/7083127893210991915?lang=en,2022-04-05 14:33:48,2022-04-05,4,Tuesday,14:33:48,2022-04-04 14:33:48,2022-04-04,2022-04-10 10:53:29.004647
8,jongraz,7082013088244895019,now venture forth and have the best weekend! 🦴🔮🦴,1648909667,20,2269,102500,1093,525000,https://www.tiktok.com/@jongraz/video/7082013088244895019?lang=en,2022-04-02 14:27:47,2022-04-02,4,Saturday,14:27:47,2022-03-28 14:27:47,2022-03-28,2022-04-10 10:53:29.004647
9,jongraz,7081623062868856110,it’s his world and we’re all just here too 🔮🦴🔮,1648818858,56,34400,1000000,6675,4400000,https://www.tiktok.com/@jongraz/video/7081623062868856110?lang=en,2022-04-01 13:14:18,2022-04-01,4,Friday,13:14:18,2022-03-28 13:14:18,2022-03-28,2022-04-10 10:53:29.004647
10,jongraz,7081257158863523114,there you have it 🔮🦴🔮,1648733664,15,6079,470400,3286,2300000,https://www.tiktok.com/@jongraz/video/7081257158863523114?lang=en,2022-03-31 13:34:24,2022-03-31,3,Thursday,13:34:24,2022-03-28 13:34:24,2022-03-28,2022-04-10 10:53:29.004647


---

### Download cover art

In [14]:
# for v,u in zip(bones_df['video_cover'], bones_df['video_id']):
#     !wget {v} -O ../assets/images/noodle_covers/{u}

---

### Weekly plays

In [15]:
weekly_plays = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_plays':sum}).reset_index()
weekly_plays.dtypes

weekstart    object
n_plays       int64
dtype: object

In [16]:
weekly_plays['weekstart'] = pd.to_datetime(weekly_plays['weekstart'])

In [28]:
alt.Chart(weekly_plays).mark_bar(width=14).encode(
    x='weekstart:T',
    y='n_plays'
).properties(width=630)

alt.Chart(...)

---

### Weekly likes

In [29]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
2,jongraz,7083871650818641198,beached and unbothered,1649342397,15,1895,68300,878,335500,https://www.tiktok.com/@jongraz/video/7083871650818641198?lang=en,2022-04-07 14:39:57,2022-04-07,4,Thursday,14:39:57,2022-04-04 14:39:57,2022-04-04,2022-04-10 10:53:29.004647
6,jongraz,7083127893210991915,another day another glaring truth,1649169228,30,1523,54700,556,270900,https://www.tiktok.com/@jongraz/video/7083127893210991915?lang=en,2022-04-05 14:33:48,2022-04-05,4,Tuesday,14:33:48,2022-04-04 14:33:48,2022-04-04,2022-04-10 10:53:29.004647
8,jongraz,7082013088244895019,now venture forth and have the best weekend! 🦴🔮🦴,1648909667,20,2269,102500,1093,525000,https://www.tiktok.com/@jongraz/video/7082013088244895019?lang=en,2022-04-02 14:27:47,2022-04-02,4,Saturday,14:27:47,2022-03-28 14:27:47,2022-03-28,2022-04-10 10:53:29.004647
9,jongraz,7081623062868856110,it’s his world and we’re all just here too 🔮🦴🔮,1648818858,56,34400,1000000,6675,4400000,https://www.tiktok.com/@jongraz/video/7081623062868856110?lang=en,2022-04-01 13:14:18,2022-04-01,4,Friday,13:14:18,2022-03-28 13:14:18,2022-03-28,2022-04-10 10:53:29.004647
10,jongraz,7081257158863523114,there you have it 🔮🦴🔮,1648733664,15,6079,470400,3286,2300000,https://www.tiktok.com/@jongraz/video/7081257158863523114?lang=en,2022-03-31 13:34:24,2022-03-31,3,Thursday,13:34:24,2022-03-28 13:34:24,2022-03-28,2022-04-10 10:53:29.004647


In [30]:
weekly_likes = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_likes':sum}).reset_index()
weekly_likes.dtypes

weekstart    object
n_likes       int64
dtype: object

In [31]:
weekly_likes['weekstart'] = pd.to_datetime(weekly_likes['weekstart'])

In [32]:
alt.Chart(weekly_likes).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_likes'
).properties(width=630)

alt.Chart(...)

---

### Weekly comments

In [33]:
weekly_comments = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_comments':sum}).reset_index()
weekly_comments.dtypes

weekstart     object
n_comments     int64
dtype: object

In [34]:
weekly_comments['weekstart'] = pd.to_datetime(weekly_comments['weekstart'])

In [35]:
alt.Chart(weekly_comments).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_comments'
).properties(width=630)

alt.Chart(...)

---

### Exports

In [36]:
weekly_plays.to_csv("../_data/weekly_plays.csv", index=False)
weekly_plays.to_csv("../assets/data/weekly_plays.csv", index=False)

In [37]:
weekly_likes.to_csv("../_data/weekly_likes.csv", index=False)
weekly_likes.to_csv("../assets/data/weekly_likes.csv", index=False)

In [38]:
weekly_comments.to_csv("../_data/weekly_comments.csv", index=False)
weekly_comments.to_csv("../assets/data/weekly_comments.csv", index=False)